# Artificial Neural Networks

ip-biocode $\cdot$ March 26, 2022

In this exercise, I build a neural network from scratch and code how it performs predictions using forward propagation. This is only to help with understanding the underlying mechanisms. All deep learning libraries have the entire training and prediction processes implemented, and so in practice you wouldn't really need to build a neural network from scratch.

## Forward Propagation

Example:
<img src="http://cocl.us/neural_network_example" alt="Neural Network Example" width=600px>

Here is a simple neural network that takes two inputs, has one hidden layer with two nodes, and an output layer with one node.

Begin by randomly initializing weights and biases in the network. We have 6 weights and 3 biases.

In [2]:
import numpy as np

weights = np.around(np.random.uniform(size=6), decimals=2)
biases = np.around(np.random.uniform(size=3), decimals=2)

print(weights)
print(biases)

[0.95 0.02 0.92 0.23 0.5  0.12]
[0.43 0.25 0.47]


Now that we have the weights and the biases defined for the network, let's compute the output for a given input, $x\_1$ and $x\_2$.


In [3]:
x_1 = 0.5 # input 1
x_2 = 0.85 # input 2

print('x1 is {} and x2 is {}'.format(x_1, x_2))

x1 is 0.5 and x2 is 0.85


Compute the wighted sum of the inputs at the first node of the hidden layer, $z_{1, 1}$. The weights are $w_1$ and $w_2$.


In [8]:
z_11 = x_1 * weights[0] + x_2 * weights[1] + biases[0]

print('The weighted sum of the inputs at the first node in the hidden layer is {}'.format(z_11))

The weighted sum of the inputs at the first node in the hidden layer is 0.9219999999999999


Next compute the weighted sum of the inputs at the second node of the hidden layer, $z_{1, 2}$. The weights are $w_3$ and $w_4$.


In [10]:
z_12 = x_1 * weights[2] + x_2 * weights[3] + biases[1]

print('The weighted sum of the inputs at the first node in the hidden layer is {}'.format(z_12))

The weighted sum of the inputs at the first node in the hidden layer is 0.9055


Next, assuming a sigmoid activation function, compute the activation of the first node, $a_{1, 1}$, in the hidden layer.
